In [2]:
from pyinjective.async_client import AsyncClient
from pyinjective.core.network import Network

import pandas as pd 


network = Network.local()
client = AsyncClient(network)


async def sub_order(client, account_address) -> None:
    subacc_list = await client.fetch_subaccounts_list(account_address)
    return subacc_list["subaccounts"]


async def order_list(client, subaccount_id,market_id) -> None:
    orders = await client.fetch_subaccount_orders_list(subaccount_id=subaccount_id, market_id=market_id)
    return orders['orders']


async def subfinder(account_address,market_id) -> None:
    all_orders =  []
    subacc_list = await sub_order(client, account_address)
    for i in subacc_list: 
        all_orders.extend(await order_list(client, i,market_id))

    df = pd.DataFrame(all_orders)
    pd.set_option('display.max_colwidth', 100)
    #print(df)
    df = df[["orderSide","subaccountId","price","quantity"]]
    df["price"] = df["price"].astype(float)/1e6
    df["quantity"] = df["quantity"].astype(float)
    df["quantity_in_$"] = df["price"] * df["quantity"]
    ## sort by quantity_in_$
    df = df.sort_values(by="quantity_in_$", ascending=False)
    df = df[["orderSide","subaccountId","price","quantity_in_$"]]
    
    return df

In [3]:
from functions import marketId_to_ticker
from IPython.display import display
import pandas as pd

tickers = ["iMSTR/USDT PERP", "iNVDA/USDT PERP", "iCOIN/USDT PERP", "iTSLA/USDT PERP", "iHOOD/USDT PERP", "iMCD/USDT PERP", "iGOOGL/USDT PERP", "TRADFI/USDT PERP", "TTI/USDT PERP"]

address = "inj1jddhqk6egwcwrwzev5wjeneht0ykmlwhcehc4z"

async def main():

    combined_df = pd.DataFrame()

    for ticker in tickers:
        market_id = await marketId_to_ticker(ticker)
        if market_id:
            print(f"Fetching orders for {ticker}...")
            df = await subfinder(account_address=address, market_id=market_id[0])
            
            df = df.iloc[:4]
            
            df['ticker'] = ticker
            
            display(df)
            
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            column_to_drop = ["orderSide","price", "quantity_in_$"]
            combined_df.drop(columns=column_to_drop, inplace=True)

    combined_df.to_csv(f"orders_{address}.csv", index=False)
    print(f"Combined CSV saved as orders_{address}.csv")

await main()

Fetching orders for iMSTR/USDT PERP...


,orderSide,subaccountId,price,quantity_in_$,ticker
21,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000037,273.39,57898.26081,iMSTR/USDT PERP
20,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000037,269.33,57038.43807,iMSTR/USDT PERP
24,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000046,271.62,8420.22000,iMSTR/USDT PERP
22,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000046,271.10,8404.10000,iMSTR/USDT PERP


Fetching orders for iNVDA/USDT PERP...


,orderSide,subaccountId,price,quantity_in_$,ticker
23,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000047,102.93,121808.70009,iNVDA/USDT PERP
1,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000032,103.61,58130.49411,iNVDA/USDT PERP
0,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000032,102.36,57429.18036,iNVDA/USDT PERP
24,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000047,103.02,9271.80000,iNVDA/USDT PERP


Fetching orders for iCOIN/USDT PERP...


...

Fetching orders for TRADFI/USDT PERP...


,orderSide,subaccountId,price,quantity_in_$,ticker
0,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000004d,514.84,102104.61332,TRADFI/USDT PERP
2,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000004d,518.52,9333.36000,TRADFI/USDT PERP
1,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000004d,514.29,9257.22000,TRADFI/USDT PERP


Fetching orders for TTI/USDT PERP...


,orderSide,subaccountId,price,quantity_in_$,ticker
1,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000036,438.03,58355.23266,TTI/USDT PERP
0,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000036,433.89,57803.69358,TTI/USDT PERP
23,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000004e,435.98,9155.58000,TTI/USDT PERP
22,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000004e,435.87,9153.27000,TTI/USDT PERP


Combined CSV saved as orders_inj1jddhqk6egwcwrwzev5wjeneht0ykmlwhcehc4z.csv
